In [82]:
git config --global user.name "Mona Lisa"

SyntaxError: invalid syntax (548772730.py, line 1)

In [1]:
# Import packages
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
from datetime import date

use the last 2yrs with a 1d interval to filter stocks, find stop-loss
- try a 59day with short interval to find optimal buy/sell 
- also can try 1d interval to find optimal buy/sell, but less optimistic on this

### look for high short term vol, low long term vol

Will need to be able to download stock data. 
- Limited by data availability. Ideally would be using intraday prices going back 2yrs but beyond 60-days that data isn't available from yfinance. 
Chosen to get recent and longer term price variability (PV) separately. Looking for low on both

In [2]:
def download_from_ticker(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    return [short_data, long_data]


def short_long_price_variability(short_long_list):
    
    """
    Calc short (2mo) and long (2yr) term price variability (price adjusted standard deviation).
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    return twomo_vol, twoyr_vol

In [32]:
#only check long if short matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())

    short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
    twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
    twomo_avg = twomo_mid_data.mean() #calculate average 
    twomo_std_dev = twomo_mid_data.std() #calculate deviation
    twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    
    if ((0.01 < twomo_vol) & (0.015 > twomo_vol)): #only check long data if short data fits, save runtime
        long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
        twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
        twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
        twoyr_avg = twoyr_mid_data.mean()
        twoyr_std_dev = twoyr_mid_data.std()
        twoyr_vol = twoyr_std_dev / twoyr_avg
    else:
        twoyr_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [41]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.025 < twoyr_vol) & (0.035 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [40]:
# testing on our poster-child, expecting low price variability
print(short_long_price_variability("GUD.TO"))

# expecting pretty high volatility on this one
print(short_long_price_variability("BBBY"))

#works on indices as well
print(short_long_price_variability("^DJI"))

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
{'Short PV': 0.01326068732974087, 'Long PV': 0.030173979158004325}
[*********************100%***********************]  1 of 1 completed
{'Short PV': -1, 'Long PV': 0.6101786749417875}
[*********************100%***********************]  1 of 1 completed
{'Short PV': -1, 'Long PV': 0.048256906290477}


It looks like BBBY has about 25x short and 20x long PV compared to GUD.TO. So the function is doing what it's supposed to.

DJI has comparable short term PV, and about 1.5x long.

GUD.TO is remarkably boring. 

Retreived ticker names from https://www.nasdaq.com/market-activity/stocks/screener

Filtered out stocks smaller than 300m mktcap, manually combined nasdaq and nyse tickers (total 3800)

Can add Canadian stocks later if desired.

In [56]:
tickers = pd.read_csv('/Users/kaigroden-gilchrist/Downloads/Personal_Project/SuccessiveSmallGains/Equity-ma' \
                      'rket-analysis/tickers.csv')

tickers = tickers.rename(columns = {"A": "Ticker"})

In [58]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/3447351491.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)


    Ticker  Short PV   Long PV
0       AA      -1.0  0.282021
1      AAC      -1.0  0.012663
2      AAN      -1.0  0.344067
3      AAP      -1.0  0.136551
4      AAT      -1.0  0.134909
..     ...       ...       ...
995    IQV      -1.0  0.103050
996     IR      -1.0  0.092336
997    IRM      -1.0  0.107275
998    IRS      -1.0  0.144160
999    IRT      -1.0  0.169820

[1000 rows x 3 columns]


In [59]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/4195299443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
1000    ISD      -1.0  0.118405
1001     IT      -1.0  0.158886
1002   ITCB      -1.0  0.229826
1003   ITGR      -1.0  0.138703
1004    ITT      -1.0  0.125410
...     ...       ...       ...
1995    WPP      -1.0  0.171037
1996    WRB      -1.0  0.144427
1997   WRBY      -1.0  0.573127
1998    WRK      -1.0  0.159328
1999    WSM      -1.0  0.165247

[1000 rows x 3 columns]


In [60]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/2941218291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
2000    WSO      -1.0  0.068931
2001    WSR      -1.0  0.125797
2002    WST      -1.0  0.207445
2003     WT      -1.0  0.093555
2004    WTI      -1.0  0.281514
...     ...       ...       ...
2995   LULU      -1.0  0.141595
2996   LUNA      -1.0  0.272257
2997   LUNG      -1.0  0.497195
2998   LUNR      -1.0  0.556944
2999   LVRA      -1.0  0.014155

[1000 rows x 3 columns]


In [61]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/1778430616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)


In [66]:
#opting to set min variability instead of volume to filter based on liquidity but could change later 
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

Empty DataFrame
Columns: [Ticker, Short PV, Long PV]
Index: []
     Ticker  Short PV   Long PV
1487    PNM  0.003570  0.032266
1855    TXO  0.024454  0.025868
     Ticker  Short PV   Long PV
2241   BCSA  0.029317  0.025597
2488   DCRD  0.058777  0.027225
2781   HONE  0.015847  0.034489
2937  LANDM  0.013412  0.032470
Empty DataFrame
Columns: [Ticker, Short PV, Long PV]
Index: []


PNM is flat in the 2yr, but not enough chop for this 
TXO is a recent IPO, not enough data.
BCSA is not flat, slowly rising. Also a recent (~1yr) IPO so not enough data
DCRD is a SPAC (explains long term PV). Recent PV too crazy (up 47% Feb 13-20)
HONE is great. Exactly what we're looking for (although ideally no dividend)
LANDM looks like it was stable, but now trending downwards in the last 6mo. Not ideal. 

Based on HONE's Long PV of 0.34, going to expand Long PV range to 0.42 to see if I can get more hits slightly higher. Moving min to 0.3 since I've screened below and don't see anything. 

In [70]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.03 < twoyr_vol) & (0.042 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [71]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/3447351491.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)


    Ticker  Short PV   Long PV
0       AA      -1.0  0.282021
1      AAC      -1.0  0.012663
2      AAN      -1.0  0.344067
3      AAP      -1.0  0.136551
4      AAT      -1.0  0.134909
..     ...       ...       ...
995    IQV      -1.0  0.103050
996     IR      -1.0  0.092336
997    IRM      -1.0  0.107275
998    IRS      -1.0  0.144160
999    IRT      -1.0  0.169820

[1000 rows x 3 columns]


In [72]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/4195299443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
1000    ISD      -1.0  0.118405
1001     IT      -1.0  0.158886
1002   ITCB      -1.0  0.229826
1003   ITGR      -1.0  0.138703
1004    ITT      -1.0  0.125410
...     ...       ...       ...
1995    WPP      -1.0  0.171037
1996    WRB      -1.0  0.144427
1997   WRBY      -1.0  0.573126
1998    WRK      -1.0  0.159328
1999    WSM      -1.0  0.165247

[1000 rows x 3 columns]


In [73]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/2941218291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
2000    WSO      -1.0  0.068931
2001    WSR      -1.0  0.125797
2002    WST      -1.0  0.207445
2003     WT      -1.0  0.093555
2004    WTI      -1.0  0.281514
...     ...       ...       ...
2995   LULU      -1.0  0.141595
2996   LUNA      -1.0  0.272257
2997   LUNG      -1.0  0.497195
2998   LUNR      -1.0  0.556944
2999   LVRA      -1.0  0.014155

[1000 rows x 3 columns]


In [74]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/1778430616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)


In [75]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

    Ticker  Short PV   Long PV
657    EQC  0.044528  0.041331
     Ticker  Short PV   Long PV
1025    JNJ  0.041363  0.038003
1487    PNM  0.003565  0.032266
     Ticker  Short PV   Long PV
2326   CBSH  0.018631  0.041063
2478   CZNC  0.014556  0.039635
2553   ENLT  0.038972  0.038501
2646   FNLC  0.015954  0.040111
2781   HONE  0.015864  0.034489
2937  LANDM  0.013412  0.032470
     Ticker  Short PV   Long PV
3059   MLYS  0.035410  0.036103
3483   SKWD  0.040144  0.036290


EQC looked good, but just broke down after a special div payment
JNJ is decent. Larger swings in price but may be enough of them not to rule it out.
CBSH looks good.
CZNC declining 1yr MA not ideal. 
ENLT recent IPO. No
FNLC looks good.
MLYS recent IPO
SKWD recent IPO.

CBSH and FNLC are towards the top end of the Long PV range, may as well check higher. 

In [76]:
#only check short if long matches
def short_long_price_variability(ticker):
    
    """
    Create a list of 2mo and 2yr datasets for the given ticker.
    Both contain ~500 observations.
    Short data comes from the last 59 days, using a 30min interval. Includes high and low price from each period. 
    Long data comes from the last 730 days, using a 1d interval. Includes high and low price from each period.
    """
    
    twoyr_start_date = datetime.datetime.now() - datetime.timedelta(730) 
    twomo_start_date = datetime.datetime.now() - datetime.timedelta(59) 
    end_date = pd.to_datetime(date.today())
    
    long_data = yf.download(ticker, start = twoyr_start_date, end = end_date, interval = "1d")[["High", "Low"]]
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twoyr_mid_data = np.array(twoyr_midpoint["Mid"])
    twoyr_avg = twoyr_mid_data.mean()
    twoyr_std_dev = twoyr_mid_data.std()
    twoyr_vol = twoyr_std_dev / twoyr_avg
    
    if ((0.042 < twoyr_vol) & (0.06 > twoyr_vol)): #only check long data if long data fits, save runtime
        short_data = yf.download(ticker, start = twomo_start_date, end = end_date, interval = "30m")[["High", "Low"]]   
        twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) #calc period midpoint
        twomo_mid_data = np.array(twomo_midpoint["Mid"]) #create array
        twomo_avg = twomo_mid_data.mean() #calculate average 
        twomo_std_dev = twomo_mid_data.std() #calculate deviation
        twomo_vol = twomo_std_dev / twomo_avg #price adjusted deviation
    else:
        twomo_vol = -1 #doesn't matter really but just setting neg
    
    #returning a dict. originally had returning a list but caused the method to run double downstream
    return {"Short PV": twomo_vol, "Long PV": twoyr_vol} 

In [77]:
tickers_0_1 = tickers[0:1000] #breaking it up into segments 

tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)

tickers_0_1_df = pd.concat([tickers_0_1[['Ticker']], tickers_0_1['PV'].apply(pd.Series)], axis=1)

print(tickers_0_1_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/3447351491.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_0_1['PV'] = tickers_0_1['Ticker'].apply(short_long_price_variability)


    Ticker  Short PV   Long PV
0       AA      -1.0  0.282021
1      AAC      -1.0  0.012663
2      AAN      -1.0  0.344067
3      AAP      -1.0  0.136551
4      AAT      -1.0  0.134909
..     ...       ...       ...
995    IQV      -1.0  0.103050
996     IR      -1.0  0.092336
997    IRM      -1.0  0.107275
998    IRS      -1.0  0.144160
999    IRT      -1.0  0.169820

[1000 rows x 3 columns]


In [78]:
tickers_1_2 = tickers[1000:2000] 

tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)

tickers_1_2_df = pd.concat([tickers_1_2[['Ticker']], tickers_1_2['PV'].apply(pd.Series)], axis=1)

print(tickers_1_2_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/4195299443.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_1_2['PV'] = tickers_1_2['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
1000    ISD      -1.0  0.118405
1001     IT      -1.0  0.158886
1002   ITCB      -1.0  0.229826
1003   ITGR      -1.0  0.138703
1004    ITT      -1.0  0.125410
...     ...       ...       ...
1995    WPP      -1.0  0.171037
1996    WRB      -1.0  0.144427
1997   WRBY      -1.0  0.573127
1998    WRK      -1.0  0.159328
1999    WSM      -1.0  0.165247

[1000 rows x 3 columns]


In [79]:
tickers_2_3 = tickers[2000:3000] 

tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)

tickers_2_3_df = pd.concat([tickers_2_3[['Ticker']], tickers_2_3['PV'].apply(pd.Series)], axis=1)

print(tickers_2_3_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/2941218291.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_2_3['PV'] = tickers_2_3['Ticker'].apply(short_long_price_variability)


     Ticker  Short PV   Long PV
2000    WSO      -1.0  0.068931
2001    WSR      -1.0  0.125797
2002    WST      -1.0  0.207445
2003     WT      -1.0  0.093555
2004    WTI      -1.0  0.281514
...     ...       ...       ...
2995   LULU      -1.0  0.141595
2996   LUNA      -1.0  0.272257
2997   LUNG      -1.0  0.497195
2998   LUNR      -1.0  0.556944
2999   LVRA      -1.0  0.014155

[1000 rows x 3 columns]


In [80]:
tickers_3_end = tickers[3000:] 

tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)

tickers_3_end_df = pd.concat([tickers_3_end[['Ticker']], tickers_3_end['PV'].apply(pd.Series)], axis=1)

print(tickers_3_end_df)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/var/folders/7c/glh90ckd7bdfn7cc0xbs4hgr0000gn/T/ipykernel_84428/1778430616.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tickers_3_end['PV'] = tickers_3_end['Ticker'].apply(short_long_price_variability)


In [81]:
tickers_0_1_df_filtered = tickers_0_1_df[tickers_0_1_df["Short PV"].between(left = 0, right = 1)]
tickers_1_2_df_filtered = tickers_1_2_df[tickers_1_2_df["Short PV"].between(left = 0, right = 1)]
tickers_2_3_df_filtered = tickers_2_3_df[tickers_2_3_df["Short PV"].between(left = 0, right = 1)]
tickers_3_end_df_filtered = tickers_3_end_df[tickers_3_end_df["Short PV"].between(left = 0, right = 1)]

print(tickers_0_1_df_filtered)
print(tickers_1_2_df_filtered)
print(tickers_2_3_df_filtered)
print(tickers_3_end_df_filtered)

    Ticker  Short PV   Long PV
24     ADC  0.019964  0.052610
30     AEE  0.020002  0.056138
87    AMCR  0.029136  0.043482
162    AVA  0.036456  0.057578
224    BDJ  0.013606  0.056681
226    BDX  0.023021  0.043537
339   BXMX  0.021189  0.056789
353    CAG  0.042531  0.059237
355    CAJ  0.015962  0.044804
359   CAPL  0.036246  0.056862
393    CHE  0.016904  0.046703
411     CL  0.031230  0.042437
430    CNA  0.020426  0.059596
456     CP  0.019901  0.053180
469   CRBG  0.028804  0.043807
550   DIAX  0.008678  0.058480
566    DNP  0.011343  0.043714
576    DPG  0.013669  0.046311
589    DUK  0.025516  0.056340
624    EIG  0.012861  0.054135
665     ES  0.031073  0.052864
716   FCPT  0.029974  0.046018
721    FEI  0.021290  0.057803
815    GHC  0.036685  0.052682
820    GIB  0.033657  0.054054
887     HE  0.017023  0.051840
892   HFRO  0.022705  0.044944
914    HMN  0.032694  0.055433
933    HRL  0.014513  0.057760
939    HTD  0.017310  0.053087
957    IBM  0.032726  0.055987
963    I

ADC looks decent. Bigger dif between highs and lows than I'd like.
AEE looks decent. Looks like a general uptrend but not too pronounced
AMCR looks nice. 
AVA is pretty good. Last 1yr better than previous.
BDJ looks to be trending down slightly. 
BDX looks nice. 
BXMX looks decent. Very cyclical around the dividend. Maybe slightly trending down. 
CAG is ok. Bigger dif between highs and lows than I'd like.
CAJ was looking good. Fell off in last 6mo. 
CAPL looks nice. Farthest end of 2yr is uptrend but since then nice. 
CHE looks good. 
CL is ok. Bigger dif between highs and lows than I'd like.
CNA not great. Wide wavelength. 
CP looks great. 
CRBG is recent IPO. No. 
DIAX looks general downtrend. No.
DNP no.
DPG looks pretty good. 
DUK not great. 
EIG looks ok. Something strange big movement in Oct 2022
ES was good. May have broken down recently. 
FCPT is ok. Bigger dif between highs and lows than I'd like
FEI is good. Far end of 2yr rising but flat since. 
GHC is ok. Not great. 
GIB no.
HE pretty good. 
HFRO no. 
HMN slight downtrend. Potentially ok.
HRL no. Strong support but hitting lower highs. 
HTD not great. 
IBM is ok. Bigger dif between highs and lows than I'd like.
IDA is ok. Bigger dif between highs and lows than I'd like.
IGD no. 
INGR not great. 
KMB is ok. Bigger dif between highs and lows than I'd like.
KMI is ok. Bigger dif between highs and lows than I'd like.
LADR is ok. Don't like the big drop in Oct 22.
MMP in a slight uptrend but still looks good. 
MSM is ok. Bigger dif between highs and lows than I'd like.
NID no. 
NNN is ok. Bigger dif between highs and lows than I'd like.
NVS is ok. Bigger dif between highs and lows than I'd like.
O is ok. Don't like the big drop in Oct 22.
OFC is ok. Don't like the big drop in Oct 22.
PB is pretty good. 
PFS is ok. Bigger dif between highs and lows than I'd like.
PM is ok. Bigger dif between highs and lows than I'd like.
POR no. 
PPL looks ok, but don't like that it's hitting lower lows since Nov 21. 
PRS no. 
SCM just broke out. No. 
SON looks pretty good. 
SPH looks pretty good. 
STZ is ok. Bigger dif between highs and lows than I'd like.
SYY is pretty good. Bigger dif between highs and lows than I'd like.
THG no. 
VOYA just broke out. No. 
WEC is ok. Bigger dif between highs and lows than I'd like.
WMT is ok. Bigger dif between highs and lows than I'd like.
WPC no. 
WTRG is ok. Bigger dif between highs and lows than I'd like.
ARCC no. 
AROW no. 
BTWN no volume. 
BWB is ok but looks too long wavelength. 
CAC no. In slight downtrend. 
CATC looks pretty good. 
CATY looks ok. 
CCNE looks pretty good. 
CGBD was flat. Breaking out now. 
CIVB no. Trending down. 
CTBI looks good. 
DCOM looks good. 
DGICA is ok. Bigger dif between highs and lows than I'd like.
EVRG is ok. Bigger dif between highs and lows than I'd like.
FRPH looked a little off far end of 2yr. Very good now. 
GGAA no. SPAC.
GLPI slight uptrend. No. 
GNTY no. 
GSBC good in the last year. 
IEP good for the past year. 
INDB pretty good for the past year. 
KDP decent. 
MDLZ decent. High amplitude. 
MGEE no. 
MOFG pretty good. 
MSBI pretty good. 
NMFC no. Was good but now too big amp. 
NWBI no. Way too big amp. 
OBIO no. Spac?
OBNK decent. Maybe slight downtrend. 
OCSL no. 
REYN big amplitude. Prob not. 
RPRX big aplitude. Prob not. 
SAFT no. Slight uptrend, large amp. 
SIRI would have been great but just crashed. 
TCPC decent. Slight downtrend. 
THFF is ok. 
TOWN is great. 
VGAS no. SAPC. 
VLGEA is decent. Don't like the big drop in Oct 22.
WABC good in the past 1.5yr.
WAFD is ok. 
WALD no. SPAC.
XEL not great. High amplitude. 

Looks like the upper end of Long PV from this batch are generally not great (with a couple exceptions), meaning we've probably found our upper limit. 

The ones that look really good from this are:
      Short     Long
HONE  0.015847  0.034489
CBSH  0.018631  0.041063
FNLC  0.015954  0.040111
AMCR  0.029136  0.043482
BDX   0.023021  0.043537
CAPL  0.036246  0.056862 
CHE   0.016904  0.046703
CP    0.019901  0.053180
CTBI  0.038825  0.048464
DCOM  0.030092  0.059724
FRPH  0.023382  0.050308
GSBC  0.016853  0.046868
IEP   0.017944  0.054363
INDB  0.023662  0.059701
TOWN  0.016220  0.050351
WABC  0.019999  0.048959

Next model prob want 0.3 < Long PV < 0.55 & 

Can use optimization to figure out buys (at least initially?)

In [ ]:
import yfinance as yf #contains stock price data
import numpy as np
import pendulum 
import matplotlib.pyplot as plt 

stock_info = yf.Ticker('TSLA').info

market_price = stock_info['regularMarketPrice']
previous_close_price = stock_info['regularMarketPreviousClose']
print('market price ', market_price)
print('previous close price ', previous_close_price)

price_history = yf.Ticker('TSLA').history(period='2y', # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
                                   interval='1mo', # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
                                   actions=False)
time_series = list(price_history['Open'])
dt_list = [pendulum.parse(str(dt)).float_timestamp for dt in list(price_history.index)]
plt.style.use('dark_background')
plt.plot(dt_list, time_series, linewidth=2)

In [ ]:
def short_long_returns_volatility_ratio(short_long_list):
    
    """
    Calc the ratio between short (2mo) and long (2yr) term daily volatility of log returns.
    Higher output means high short term vol relative to long term.
    
    Literature uses log returns to calculate volatility. For my purposes I'm not sure this is necessary.
    I only care about deviation in price so a more simple approach should suffice. 
    """
    
    short_data = short_long_list[0]
    long_data = short_long_list[1]
    
    twomo_midpoint = short_data.assign(Mid = lambda col: (col.High + col.Low)/2) 
    twomo_log_returns = twomo_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twomo_std_dev = np.array(twomo_log_returns["log_returns"])[1:].std()
    twomo_vol = twomo_std_dev * (59**0.5) 
    
    twoyr_midpoint = long_data.assign(Mid = lambda col: (col.High + col.Low)/2)
    twoyr_log_returns = twoyr_midpoint.assign(log_returns = lambda x: np.log(x.Mid / x.Mid.shift()))
    twoyr_std_dev = np.array(twoyr_log_returns["log_returns"])[1:].std()
    twoyr_vol = twoyr_std_dev * (730**0.5)
    
    return twomo_vol, twoyr_vol